In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
rm generated_bees_epoch_*.png

In [1]:

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.models import vgg16
from PIL import Image

# ----------------------------
# Hyperparameters & Settings
# ----------------------------
img_size       = 64
channels       = 3
latent_dim     = 100
batch_size     = 32
num_epochs     = 50000
learning_rate  = 5e-5
warmup_epochs  = 3000    # KL warm-up duration
perc_weight    = 0.01   # weight for perceptual loss
device         = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ----------------------------
# Dataset
# ----------------------------
class BeeDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]
        # Use all images in directory
        self.img_files = files

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        path = os.path.join(self.img_dir, self.img_files[idx])
        img = Image.open(path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

# ----------------------------
# VAE Model
# ----------------------------
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(channels, 32, 4, 2, 1),  # 64 → 32
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1),        # 32 → 16
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1),       # 16 → 8
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * 8 * 8, 1024),
            nn.ReLU()
        )
        self.fc_mu     = nn.Linear(1024, latent_dim)
        self.fc_logvar = nn.Linear(1024, latent_dim)
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * 8 * 8),
            nn.ReLU(),
            nn.Unflatten(1, (128, 8, 8)),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),  # 8 → 16
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),   # 16 → 32
            nn.ReLU(),
            nn.ConvTranspose2d(32, channels, 4, 2, 1),  # 32 → 64
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h      = self.encoder(x)
        mu     = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        z      = self.reparameterize(mu, logvar)
        x_recon= self.decoder(z)
        return x_recon, mu, logvar

# ----------------------------
# Loss Functions
# ----------------------------
def vae_loss(x_recon, x, mu, logvar):
    recon = F.binary_cross_entropy(x_recon, x, reduction='sum')
    kl    = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon, kl

# Perceptual loss using VGG
vgg = vgg16(pretrained=True).features[:16].to(device).eval()
for param in vgg.parameters():
    param.requires_grad = False

normalize_for_vgg = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

def perceptual_loss(x_recon, x):
    # Normalize both
    xr = normalize_for_vgg(x_recon)
    xg = normalize_for_vgg(x)
    feat_r = vgg(xr)
    feat_g = vgg(xg)
    return F.l1_loss(feat_r, feat_g)

# ----------------------------
# Data & Model Setup
# ----------------------------
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

dataset = BeeDataset(
    img_dir='/content/drive/MyDrive/DiT_Project/dataset/Bee_1000Data',
    transform=transform
)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

model     = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# ----------------------------
# Training Loop
# ----------------------------
for epoch in range(1, num_epochs + 1):
    model.train()
    total_recon = 0
    total_kl    = 0
    total_perc  = 0

    for x in loader:
        x = x.to(device)
        x_recon, mu, logvar = model(x)

        # compute losses
        recon_loss = vae_loss(x_recon, x, mu, logvar)[0]
        kl_loss    = vae_loss(x_recon, x, mu, logvar)[1]
        perc_loss  = perceptual_loss(x_recon, x)

        # KL warm-up
        kl_w = min(1.0, epoch / warmup_epochs)
        loss = (recon_loss + kl_w * kl_loss + perc_weight * perc_loss) / x.size(0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_recon += recon_loss.item()
        total_kl    += kl_loss.item()
        total_perc  += perc_loss.item()

    scheduler.step()

    avg_recon = total_recon / len(dataset)
    avg_kl    = total_kl    / len(dataset)
    avg_perc  = total_perc  / len(dataset)
    print(f'Epoch {epoch}/{num_epochs} | Recon: {avg_recon:.4f} | KL: {avg_kl:.4f} '
          f'| Perc: {avg_perc:.4f} | KL_w: {kl_w:.2f}')

    # Save samples every 100 epochs
    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
            z       = torch.randn(16, latent_dim, device=device)
            samples = model.decoder(z)
            save_image(samples, f'generated_bees_epoch_{epoch}.png', nrow=4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 231MB/s]


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DiT_Project/dataset/Bee_1000Data'

In [7]:
import os
import torch
from torchvision.utils import save_image

# 1. 配置
output_dir = 'generated_1000_highres'
os.makedirs(output_dir, exist_ok=True)

n_samples    = 1000         # 总共要生成的张数
batch_size_g = 50           # 每次采样的 batch，调大可以更快，但要看显存

# 2. 加载模型（如果刚训练完就在同一脚本里就跳过这步）
# model = VAE(...)         # 定义网络结构
# model.load_state_dict(torch.load('best_checkpoint.pth')['model_state_dict'])
model.eval()
model.to(device)

# 3. 批量生成
with torch.no_grad():
    count = 0
    while count < n_samples:
        # 本次实际生成数量，处理最后一次可能不够 batch_size_g 的情况
        bs = min(batch_size_g, n_samples - count)
        z = torch.randn(bs, latent_dim, device=device)
        imgs = model.decoder(z)  # 输出 [bs, C, H, W]，值在 [0,1]

        # 保存每张
        for i in range(bs):
            idx = count + i
            save_image(
                imgs[i],
                os.path.join(output_dir, f'img_{idx:04d}.png')
            )
        count += bs
        print(f'Generated {count}/{n_samples}')

print('All done! Images saved to', output_dir)

Generated 50/1000
Generated 100/1000
Generated 150/1000
Generated 200/1000
Generated 250/1000
Generated 300/1000
Generated 350/1000
Generated 400/1000
Generated 450/1000
Generated 500/1000
Generated 550/1000
Generated 600/1000
Generated 650/1000
Generated 700/1000
Generated 750/1000
Generated 800/1000
Generated 850/1000
Generated 900/1000
Generated 950/1000
Generated 1000/1000
All done! Images saved to generated_1000_highres
